# Plan your trip with Kayak



Contents
--------
1. [Utilities](#utils)
    A. [Scraping utilities](#scraping_utils)
    B. [Input-output utilities](#io_utils)
2. [Perspectives](#perspectives)

## <a name="utils"></a>Utililties

This section introduces some utilities to be used later in the project. These can be seen as the different modules of a python package.

### <a name="scraping_utils"></a>Scraping utilities

This section defines utilities pertaining to data scraping. to:
- Get geographic coordinates for a selected destination
- Get weather data at given geographic coordinates
- Get information about hotels in the destination surroundings

In [15]:
import asyncio

import requests


def get_coords(destination: str)-> tuple[float, float]:
    """
    Get the geographic coordinates of 
    """


,url,name,gender,culture,born,died,titles,aliases,father,mother,spouse,allegiances,books,povBooks,tvSeries,playedBy
1,https://anapioficeandfire.com/api/characters/1,,Female,Braavosi,,,[],[The Daughter of the Dusk],,,,[],[https://anapioficeandfire.com/api/books/5],[],[],[]
2,https://anapioficeandfire.com/api/characters/2,Walder,Male,,,,[],[Hodor],,,,[https://anapioficeandfire.com/api/houses/362],"[https://anapioficeandfire.com/api/books/1, ht...",[],"[Season 1, Season 2, Season 3, Season 4, Seaso...",[Kristian Nairn]
3,https://www.anapioficeandfire.com/api/characte...,,Male,,,,[],[Lamprey],,,,[https://www.anapioficeandfire.com/api/houses/15],[https://www.anapioficeandfire.com/api/books/3],[],[],[]
4,https://www.anapioficeandfire.com/api/characte...,,Female,Braavosi,,,[],[The Merling Queen],,,,[],[https://www.anapioficeandfire.com/api/books/5...,[],[],[]
5,https://www.anapioficeandfire.com/api/characte...,,Male,,,,[],[Old Crackbones],,,,[],[https://www.anapioficeandfire.com/api/books/5],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,https://anapioficeandfire.com/api/characters/146,Arwyn Frey,Female,,In 285 AC,,[],[],,,,[https://anapioficeandfire.com/api/houses/143],"[https://anapioficeandfire.com/api/books/2, ht...",[],[],[]
147,https://anapioficeandfire.com/api/characters/147,Arwyn Oakheart,Female,,,,[Lady of Old Oak],[],,,,[https://anapioficeandfire.com/api/houses/286],"[https://anapioficeandfire.com/api/books/1, ht...",[],[],[]
148,https://anapioficeandfire.com/api/characters/148,Arya Stark,Female,Northmen,"In 289 AC, at Winterfell",,[Princess],"[Arya Horseface, Arya Underfoot, Arry, Lumpyfa...",,,,[https://anapioficeandfire.com/api/houses/362],[],"[https://anapioficeandfire.com/api/books/1, ht...","[Season 1, Season 2, Season 3, Season 4, Seaso...",[Maisie Williams]
149,https://anapioficeandfire.com/api/characters/149,Arys Oakheart,Male,Reach,At Old Oak,"In 300 AC, at the Greenblood",[Ser],[],,,,[https://anapioficeandfire.com/api/houses/286],"[https://anapioficeandfire.com/api/books/1, ht...",[https://anapioficeandfire.com/api/books/5],[],[]


In [27]:
import requests
import pandas as pd


r = requests.get(f"https://api.coingecko.com/api/v3/coins/markets",
                 headers={'accept': 'application/json'},
                 params={'vs_currency': 'usd'})
r

<Response [200]>

In [28]:
r.text

'[{"id":"bitcoin","symbol":"btc","name":"Bitcoin","image":"https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400","current_price":61852,"market_cap":1223022139670,"market_cap_rank":1,"fully_diluted_valuation":1299422620820,"total_volume":26026495472,"high_24h":63130,"low_24h":61745,"price_change_24h":-713.5546661440167,"price_change_percentage_24h":-1.1405,"market_cap_change_24h":-13505049143.988037,"market_cap_change_percentage_24h":-1.09218,"circulating_supply":19765290.0,"total_supply":21000000.0,"max_supply":21000000.0,"ath":73738,"ath_change_percentage":-15.99171,"ath_date":"2024-03-14T07:10:36.635Z","atl":67.81,"atl_change_percentage":91253.62041,"atl_date":"2013-07-06T00:00:00.000Z","roi":null,"last_updated":"2024-10-09T13:29:52.257Z"},{"id":"ethereum","symbol":"eth","name":"Ethereum","image":"https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628","current_price":2428.21,"market_cap":292433572189,"market_cap_rank":2,"fully_dilut